In [2]:
from google.colab import drive 
drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [27]:
import pandas as pd
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import layers, models, backend as K, callbacks
from keras.layers import LSTM
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import layers, models, backend as K, callbacks
from keras.layers import LSTM
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn import preprocessing
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from keras import regularizers
from keras.layers import TimeDistributed



In [ ]:
stock_market_df = pd.read_csv('/content/gdrive/MyDrive/Stock_market_label_data.csv')

In [61]:
stock_market_df = stock_market_df[(stock_market_df['Match'] != '0') & (stock_market_df['Match'] != 'NAN')]

In [ ]:
stock_market_df.tail()

In [63]:
stock_market_df['Match'] = stock_market_df['Match'].replace(['-1'], '0')

In [64]:
stock_market_df.drop(columns=['Company_Name', 'Unnamed: 0', 'Date', 'Name'], inplace=True)
stock_market_df['Match'].replace('NAN', -10, inplace=True)

In [ ]:
stock_market_df.fillna(0)

In [68]:
y = stock_market_df.iloc[:, -1:].values

In [69]:
y= y.reshape(y.shape[0], y.shape[1], 1)

In [78]:
y

array([[[1]],

       [[1]],

       [[1]],

       ...,

       [['0']],

       [['0']],

       [['0']]], dtype=object)

In [71]:
stock_market_df_1 = stock_market_df.drop(columns=['Match'])
X=stock_market_df_1.values
columns = stock_market_df_1.columns

In [ ]:
stock_market_df_1.head()

In [ ]:
norm = MinMaxScaler()
X_norm = norm.fit_transform(X)
X_norm = pd.DataFrame(X_norm, columns = columns)
X_norm.head()

In [ ]:
X = X_norm.values.reshape(X_norm.shape[0],X_norm.shape[1],1)
X

In [79]:
y.shape

(15863, 1, 1)

In [82]:
train_X,test_X,train_y,test_y= train_test_split(X,y,test_size = 0.30,random_state = 42)

In [41]:
train_X = train_X.reshape(train_X.shape[0],train_X.shape[1],1)


In [42]:
train_X.shape

(11104, 1, 82)

In [ ]:
train_X = np.asarray(train_X).astype(np.float32)
train_X

In [86]:
test_X = test_X.reshape(test_X.shape[0],1,-1)

In [ ]:
test_X = np.asarray(test_X).astype(np.float32)
test_X

In [ ]:
train_y = np.asarray(train_y).astype(np.float32)
train_y

In [ ]:
test_y = np.asarray(test_y).astype(np.float32)
test_y

In [90]:
train_X.shape

(11104, 82, 1)

In [91]:
test_y.shape

(4759, 1, 1)

In [92]:
train_X.shape

(11104, 82, 1)

In [93]:
win_length = 60
batch_size = 32
num_features = 82

model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(win_length, num_features), return_sequences=True))
model.add(TimeDistributed(layers.Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = [tf.keras.metrics.AUC()])
  

In [170]:
model.fit(train_X, train_y, epochs=100, batch_size=2, verbose=1, validation_data=(test_X, test_y))

Epoch 1/100


TypeError: ignored

# **Old method**

In [ ]:
stock_market_df = pd.read_csv('/content/gdrive/MyDrive/Stock_market_label_data.csv')
stock_market_df['Match'] = stock_market_df['Match'].replace(['-1'], '0')
stock_market_df.drop(columns=['Company_Name', 'Unnamed: 0', 'Date', 'Name'], inplace=True)
stock_market_df.fillna(0)
stock_market_df = stock_market_df[: 15862]

In [120]:
stock_market_df_1 = stock_market_df.drop(columns=['Match'])
X=stock_market_df_1.values
columns = stock_market_df_1.columns

In [ ]:
norm = MinMaxScaler()
X_norm = norm.fit_transform(X)
X_norm = pd.DataFrame(X_norm, columns = columns)
X_norm.head()

In [122]:
df1 = stock_market_df['Match']
df1 = pd.concat([df1, df1.shift(1)], axis=1)
df1.dropna(inplace=True)

In [123]:
df1 = df1.astype(float)

In [124]:
values = df1.values

In [125]:
values

array([[1., 1.],
       [1., 1.],
       [1., 1.],
       ...,
       [1., 1.],
       [0., 1.],
       [0., 0.]])

In [129]:
encoder = preprocessing.LabelEncoder()
test_labels = to_categorical(encoder.fit_transform(values[:,-1]))
train_X,test_X,train_y,test_y= train_test_split(values[:,0], test_labels,test_size = 0.30,random_state = 42)

In [130]:
train_X = train_X.reshape(train_X.shape[0],1,1)
test_X = test_X.reshape(test_X.shape[0],1,1)

In [56]:
n_batch = 1
n_epoch = 10
n_neurons = 100

In [ ]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(n_neurons, batch_input_shape=(n_batch, train_X.shape[1],train_X.shape[2]), stateful=True),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(100, activation = 'relu',kernel_regularizer=regularizers.l2(0.0001)),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(train_X,train_y,validation_data=(test_X, test_y),epochs=n_epoch, batch_size=n_batch, verbose=1,shuffle= False)

# **Old method**

In [154]:
stock_market_df = pd.read_csv('/content/gdrive/MyDrive/Stock_market_label_data.csv')
stock_market_df['Match'] = stock_market_df['Match'].replace(['-1'], '0')
stock_market_df.drop(columns=['Company_Name', 'Unnamed: 0', 'Date', 'Name'], inplace=True)
stock_market_df.fillna(0)
stock_market_df = stock_market_df[: 15862]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [155]:
stock_market_df_1 = stock_market_df.drop(columns=['Match'])
X=stock_market_df_1.values
columns = stock_market_df_1.columns

In [ ]:
norm = MinMaxScaler()
X_norm = norm.fit_transform(X)
X_norm = pd.DataFrame(X_norm, columns = columns)
X_norm.head()

In [157]:
y = stock_market_df['Match']

In [158]:
X = np.array(X_norm.values)
y = np.array(y.values)

In [159]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, shuffle=False
)

In [160]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1, shuffle=False
)

In [169]:
X_test.shape

(1587, 82)

In [161]:
X_valid.shape

(1428, 82)

In [162]:
win_length = 60
batch_size = 32
num_features = 82
train_generator = TimeseriesGenerator(np.array(X_train), np.array(y_train),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

test_generator = TimeseriesGenerator(np.array(X_test), np.array(y_test),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

valid_generator = TimeseriesGenerator(np.array(X_valid), np.array(y_valid),
                                     length=win_length,
                                     sampling_rate=1,
                                     batch_size=batch_size)

In [163]:
X_train.shape

(12847, 82)

In [130]:
model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(win_length, num_features), return_sequences=True))
model.add(TimeDistributed(layers.Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = [tf.keras.metrics.AUC()])
  

In [137]:
model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(win_length, num_features), return_sequences=True))
model.add(TimeDistributed(layers.Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = [tf.keras.metrics.AUC()])


In [164]:
model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(win_length, num_features), return_sequences=True))
model.add(TimeDistributed(layers.Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = [tf.keras.metrics.AUC()])


In [152]:
train_generator.batch_size

32

In [165]:
checkpoint_filepath = '/content/sample_data/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max')

model.fit_generator(train_generator,
                   epochs=10,
                   validation_data=valid_generator,
                   shuffle=False, callbacks=[model_checkpoint_callback])

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10


ValueError: ignored